In [1]:
import numpy as np
from pathlib import Path
import sys
import random

In [2]:
class TreeNode:
    def __init__(self, level, V, W, taken):
        self.level = level # niveau dans l’arbre de recherche >=0
        self.V = V # valeur de la solution courante
        self.W = W # poids de la solution courante
        self.taken = taken # liste des index d’objets de la solution courante
    def __str__(self):
        return str((self.level, self.V, self.W, self.taken))

In [5]:
class Genetic:
    def __init__(self, cap, values,weights,taille_pop=20,coef_filtre=0.3,pourcent=0.5,epoch=10): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.taille_pop=taille_pop
        self.coef_filtre=coef_filtre
        self.proba=pourcent
        self.population=[]
        self.epoch=epoch
        
    def genere_pop(self):
        if  len(self.population)==0:
            for i in range(self.taille_pop):
                individus = [ bool(np.random.randint(0, 2)) for _ in range(len(self.values))]
                while self.evaluate_weights(individus) > self.capacity:
                       individus = [ bool(np.random.randint(0, 2) )for _ in range(len(self.values))]
                self.population.append(individus)
        else:
           self.mutation() 

    def filtre(self):
        value=[np.sum(self.values[p]) for p in self.population]
        value=np.argsort(value)
        nb_filtre=(self.coef_filtre*self.taille_pop)/100
        self.population=self.population[nb_filtre:]
    
    def cross_over(self):
        index=np.random.randint(0,self.taille_pop,2)
        parent=np.array(self.population)[index]
        taille_div=np.random.randint(1,self.taille_pop)
        nb_genere=100-((self.coef_filtre*self.taille_pop)/100)
        childs=[]
        for i in range(int(nb_genere)):
            index=np.random.randint(0,self.taille_pop,2)
            parent=np.array(self.population)[index]
            taille_div=np.random.randint(1,len(parent))  
            child=np.concatenate([parent[0][0:taille_div],parent[1][taille_div:]])
            while self.evaluate_weights(child) > self.capacity:
                taille_div=np.random.randint(1,len(parent))  
                child=np.concatenate([parent[0][0:taille_div],parent[1][taille_div:]])
            
            childs.append(child)
        return childs

        
    def mutation(self):
        childs=self.cross_over()
        p=np.random.uniform(0,1)
        for child in childs:
            p=np.random.uniform(0,1)
            if p>=self.proba:
                index=np.random.randint(0,len(self.values))
                child[index]=1-child[index]
                while self.evaluate_weights(child) > self.capacity:
                    index=np.random.randint(0,len(self.values))
                    child[index]=1-child[index]
            self.population.append(child)

    def evaluate_values(self, solution):
        return sum(self.values[solution])
    
    def evaluate_weights(self, solution):
        return sum(self.weights[solution])
    
    def solve(self):
        for i in range(self.epoch):
            self.genere_pop()
            self.mutation()
            self.genere_pop()
        index_solution=np.argmax([self.evaluate_values(s) for s in self.population])
        solution=np.array(self.population)[index_solution]
        return solution,self.evaluate_values(solution),self.evaluate_weights(solution)

In [6]:
capacite = 10
values = [40, 50, 100, 95, 30]
weights = [2, 3, 1, 5, 3]
kpp = Genetic(capacite, np.array(values),np.array(weights))
valeur =kpp.solve()

KeyboardInterrupt: 

In [22]:
#print(f'objets -> {solution}')
print(f'Gain -> {valeur}')
# print(f'Poids -> {poid}')

Gain -> 245


In [28]:
def execute_all(directory):
    p=Path('./instances_01_KP/'+directory)
    print('Fichier            Poids')
    for f in p.iterdir():
        
        first_row=np.loadtxt(f.absolute(),max_rows=1)
        fich=np.loadtxt(f.absolute(),max_rows=int(first_row[0])+1)
        kpp = Genetic_(fich[0][1],fich[1:,1],fich[1:,0])
        valeur =kpp.solve()
        print(f'{f.name}-> {valeur}')
        

In [33]:
execute_all('low-dimensional')

Fichier            Poids
f2_l-d_kp_20_878-> 1042.0
f8_l-d_kp_23_10000-> 9777.0
f4_l-d_kp_4_11-> 4.0
f10_l-d_kp_20_879-> 1042.0
f5_l-d_kp_15_375-> 647.3952449999999
f3_l-d_kp_4_20-> 11.0
f1_l-d_kp_10_269-> 431.0
f6_l-d_kp_10_60-> 80.0
f7_l-d_kp_7_50-> 26.0
f9_l-d_kp_5_80-> 68.0


In [21]:
class Genetic_:
    def __init__(self, cap, valeurs, poids, taille_pop=50, coef_filtre=50, proba=0.1):
        self.capacity = cap
        self.valeurs = valeurs
        self.poids = poids
        self.n = len(valeurs)
        self.taille_pop = taille_pop
        self.coef_filtre = coef_filtre
        self.proba = proba
        self.population = []

    def total_weight(self, individu):
        return sum(self.poids[i] for i in range(self.n) if individu[i] == 1)

    def fitness(self, individu):
        return sum(self.valeurs[i] for i in range(self.n) if individu[i] == 1)

    def genere_pop(self):
        while len(self.population) < self.taille_pop:
            individu = [random.randint(0, 1) for _ in range(self.n)]
            while self.total_weight(individu) > self.capacity:
                individu[random.randint(0, self.n - 1)] = 0
            self.population.append(individu)

    def filtre(self):
        self.population.sort(key=self.fitness, reverse=True)
        k = int(self.taille_pop * self.coef_filtre / 100)
        self.population = self.population[:self.taille_pop - k]

    def cross_over(self):
        enfants = []
        k = int(self.taille_pop * self.coef_filtre / 100)
        for _ in range(k):
            p1, p2 = random.sample(self.population, 2)
            point = random.randint(1, self.n - 1)
            enfant = p1[:point] + p2[point:]
            while self.total_weight(enfant) > self.capacity:
                enfant[random.randint(0, self.n - 1)] = 0
            enfants.append(enfant)
        return enfants

    def mutation(self, enfants):
        for enfant in enfants:
            for i in range(self.n):
                if random.random() < self.proba:
                    enfant[i] = 1 - enfant[i]
            while self.total_weight(enfant) > self.capacity:
                enfant[random.randint(0, self.n - 1)] = 0
        self.population.extend(enfants)

    def solve(self, generations=500):
        self.genere_pop()
        for _ in range(generations):
            self.filtre()
            enfants = self.cross_over()
            self.mutation(enfants)
        best = max(self.population, key=self.fitness)
        return self.fitness(best)

In [22]:
execute_all('large_scale')

Fichier        Optimum     Poids
knapPI_3_100_1000_1-> 895.0


KeyboardInterrupt: 